[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/intel-analytics/BigDL/blob/main/python/nano/notebooks/pytorch/tutorial/pytorch_inference_onnx.ipynb)

# Apply ONNXRuntime Acceleration on Inference Pipeline

### Introduction
In this notebook we will describe how to apply ONNXRuntime Acceleration on inference pipeline with the APIs delivered by BigDL-Nano in 4 simple steps.

### Step 0: Prepare Environment
Before you start with APIs delivered by bigdl-nano, you have to make sure BigDL-Nano is correctly installed for PyTorch.

We recommend to run below commands, especially `source bigdl-nano-init` before jupyter kernel is started, or some of the optimizations may not take effect.

In [ ]:
# nightly bulit version
!pip install --pre --upgrade bigdl-nano[pytorch]
# set env variables
!source bigdl-nano-init

Before you start with onnxruntime accelerator, you need to install some onnx packages as follows to set up your environment with ONNXRuntime acceleration.

In [ ]:
!pip install onnx onnxruntime

### Step 1: Load Data
Here we use the Oxford-IIIT Pet Dataset, you could find more information from [here](https://www.robots.ox.ac.uk/~vgg/data/pets/). Note that the dataset is hosted on https://www.robots.ox.ac.uk, and its server sometimes may fail. If you got an `HTTP 500` error when downloading the dataset, it may be because that their server just failed. You could wait for some time and try again.

In [11]:
import os
import torch
from torchvision.io import read_image
from torchvision import transforms
from torchvision.datasets import OxfordIIITPet
from torch.utils.data.dataloader import DataLoader

train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.RandomCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ColorJitter(brightness=.5, hue=.3),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
val_transform = transforms.Compose([transforms.Resize([224, 224]), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
# Apply data augmentation to the tarin_dataset
train_dataset = OxfordIIITPet(root = "/tmp/data", transform=train_transform, download=True)
val_dataset = OxfordIIITPet(root="/tmp/data", transform=val_transform)
# obtain training indices that will be used for validation
indices = torch.randperm(len(train_dataset))
val_size = len(train_dataset) // 4
train_dataset = torch.utils.data.Subset(train_dataset, indices[:-val_size])
val_dataset = torch.utils.data.Subset(val_dataset, indices[-val_size:])
# prepare data loaders
train_dataloader = DataLoader(train_dataset, batch_size=32)

DEV_RUN = bool(os.environ.get('DEV_RUN', False))

### Step 2: Custom Model
Regarding the model, we used pretrained torchvision.models.resnet18. More details, please refer to [here](https://pytorch.org/vision/0.12/generated/torchvision.models.resnet18.html?highlight=resnet18).

If you run the notebook on Colab, you may meet `OSError:... undefined symbol...` error regarding `torchtext` when importing `Trainer` from `bigdl.nano.pytorch`. To solve this, you could try to downgrade `torchtext` to the compatible version with `torch`. You could refer to the table [here](https://pypi.org/project/torchtext/) for which verison of `torchtext` you should install.

Our nightly-built `bigdl-nano[pytorch]` is currently based on `torch==1.11.0`. So you could run the following command to solve this problem:

In [ ]:
# Please ignore this if you do not met problem with torchtext
!pip install torchtext==0.12.0 

In [12]:
import torch
from torchvision.models import resnet18
from bigdl.nano.pytorch import Trainer
model_ft = resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

# Here the size of each output sample is set to 37.
model_ft.fc = torch.nn.Linear(num_ftrs, 37)
loss_ft = torch.nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Compile our model with loss function, optimizer.
model = Trainer.compile(model_ft, loss_ft, optimizer_ft)
trainer = Trainer(max_epochs=5,
                  fast_dev_run=DEV_RUN) # Run model quickly in test
trainer.fit(model, train_dataloaders=train_dataloader)

# Inference/Prediction
x = torch.stack([val_dataset[0][0], val_dataset[1][0]])
model_ft.eval()
y_hat = model_ft(x)
y_hat.argmax(dim=1)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 11.2 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.782    Total estimated model params size (MB)


Epoch 4: 100%|██████████| 87/87 [00:42<00:00,  2.07it/s, loss=0.339, v_num=18]  


tensor([2, 6])

### Step 3: ONNXRuntime Acceleration
trace your model as an ONNXRuntime model using `Trainer.trace()`

In [13]:
from bigdl.nano.pytorch import Trainer
ort_model = Trainer.trace(model_ft, accelerator="onnxruntime", input_sample=torch.rand(1, 3, 224, 224))

The usage is almost the same with any PyTorch module

In [14]:
y_hat = ort_model(x)
y_hat.argmax(dim=1)

tensor([2, 6])